In [27]:
import numpy as np
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [28]:
bankdata = pd.read_csv("C:/Users/user/Desktop/HW7/data.csv")

In [29]:
bankdata.shape

(1349, 8)

In [30]:
bankdata.head()

,Unnamed: 0,buying,maint,doors,persons,lug_boot,safety,class
0,1,4,1,4,2,1,1,2
1,2,4,1,4,3,3,2,2
2,3,4,3,2,2,3,1,1
3,4,4,3,2,2,2,2,2
4,5,4,3,2,2,2,3,2


In [31]:
bankdata.drop(['Unnamed: 0'], axis=1, inplace=True)

In [32]:
bankdata.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,4,1,4,2,1,1,2
1,4,1,4,3,3,2,2
2,4,3,2,2,3,1,1
3,4,3,2,2,2,2,2
4,4,3,2,2,2,3,2


In [33]:
#features
X = bankdata.drop('class', axis=1)
#class
y = bankdata['class']

In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
#we keep 30% random examples for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [35]:
#we scale data in the range [-1,1]
from sklearn.preprocessing import MinMaxScaler
scaling=MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train=scaling.transform(X_train)
X_test=scaling.transform(X_test)

In [36]:
#X_train.shape, y_train.shape

In [37]:
#X_test.shape, y_test.shape

In [38]:
#we balance the classes
learning_alg = SVC(kernel='linear', C = 1.0)
#C is a valuation of "how badly" we want to properly classify

In [39]:
cv_performance= cross_val_score(learning_alg, X_train,y_train,cv=5)
test_performance=learning_alg.fit(X_train,y_train).score(X_test,y_test)
#cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
#scores = cross_val_score(clf, X, y, cv=5)

In [40]:
#Cross-validation accuracy scor:%0.3f,test accuracy score:%0.3f'
print(np.mean(cv_performance),test_performance)

0.6907182258246087 0.6765432098765433


In [41]:
y_pred = learning_alg.predict(X_test)

In [42]:
from sklearn.metrics import classification_report, confusion_matrix
print("__________linear__________")
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

__________linear__________
[[ 78  73]
 [ 58 196]]
              precision    recall  f1-score   support

           1       0.57      0.52      0.54       151
           2       0.73      0.77      0.75       254

    accuracy                           0.68       405
   macro avg       0.65      0.64      0.65       405
weighted avg       0.67      0.68      0.67       405



In [43]:
from sklearn.svm import SVC
learning_alg = SVC(kernel='poly', degree=8, C=1)
learning_alg.fit(X_train, y_train)

SVC(C=1, degree=8, kernel='poly')

In [44]:
y_pred = learning_alg.predict(X_test)

In [45]:
from sklearn.metrics import classification_report, confusion_matrix
print("__________poly__________")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

__________poly__________
[[ 67  84]
 [ 39 215]]
              precision    recall  f1-score   support

           1       0.63      0.44      0.52       151
           2       0.72      0.85      0.78       254

    accuracy                           0.70       405
   macro avg       0.68      0.65      0.65       405
weighted avg       0.69      0.70      0.68       405



In [46]:
from sklearn.svm import SVC
learning_alg = SVC(kernel='rbf', random_state=0, gamma=.01, C=1)
learning_alg.fit(X_train, y_train)

SVC(C=1, gamma=0.01, random_state=0)

In [47]:
y_pred = learning_alg.predict(X_test)

In [48]:
from sklearn.metrics import classification_report, confusion_matrix
print("__________rbf__________")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

__________rbf__________
[[ 75  76]
 [ 52 202]]
              precision    recall  f1-score   support

           1       0.59      0.50      0.54       151
           2       0.73      0.80      0.76       254

    accuracy                           0.68       405
   macro avg       0.66      0.65      0.65       405
weighted avg       0.68      0.68      0.68       405



In [49]:
#finding optimal number for C which is our regularization
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1,5, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1,5, 10, 100, 1000]}]

scores = ['precision', 'recall']


In [50]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

# Tuning hyper-parameters for precision

# Tuning hyper-parameters for recall



In [51]:
learning_alg = GridSearchCV(SVC(), tuned_parameters, scoring='%s_macro' % score)
learning_alg.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 5, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 5, 10, 100, 1000], 'kernel': ['linear']}],
             scoring='recall_macro')

In [54]:
print("Best parameters set found on development set:")
print()
print(learning_alg.best_params_)
print()
print("Grid scores on development set:")
print()
means = learning_alg.cv_results_['mean_test_score']

stds = learning_alg.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, learning_alg.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, learning_alg.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.500 (+/-0.000) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}

0.500 (+/-0.000) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}

0.616 (+/-0.051) for {'C': 5, 'gamma': 0.001, 'kernel': 'rbf'}

0.500 (+/-0.000) for {'C': 5, 'gamma': 0.0001, 'kernel': 'rbf'}

0.657 (+/-0.044) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

0.500 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

0.668 (+/-0.052) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

0.657 (+/-0.047) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}

0.766 (+/-0.067) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

0.663 (+/-0.062) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

0.666 (+/-0.067) for {'C': 1, 'kernel': 'linear'}

0.665 (+/-0.066) for {'C': 5, 'kernel': 'linear'}

0.665 (+/-0.066) for {'C': 10, 'kernel': 'linear'}

0.665 (+/-0.066) for {'C': 100, 'kernel': 'linear'}

0.664 (+/-